# World Model Core
*Sean Steinle, Kiya Aminfar*

This notebook walks through the core aspects of world models, developing crucial pieces of code sequentially. Not that this code isn't meant for scale -- instead, this is for a demonstration of how we developed the code that we did.

## Table of Contents
1. [Collecting Rollout Data](#Collecting-Rollout-Data)
2. [Training the VAE](#Training-the-VAE)
3. [Training the MDN-RNN](#Training-the-MDN-RNN)
    - [Prepping Rollout Data for the MDN-RNN](#Prepping-Rollout-Data-for-the-MDN-RNN)
    - [Core Training](#Core-Training)
4. [Training the Controller](#Training-the-Controller)
5. [Early Results](#Early-Results)

In [1]:
import gymnasium as gym
import matplotlib.pyplot as plt
import os
import numpy as np

## Collecting Rollout Data

In [2]:
#Let's begin by creating an instance of our humanoid environment and checking out what basic observations look like.
env = gym.make('Humanoid-v5', render_mode="rgb_array")
obs, info = env.reset()

In [3]:
obs.shape, obs

((348,),
 array([ 1.40341577e+00,  1.00747925e+00,  9.73546305e-04,  1.06873046e-03,
         9.89902476e-04, -6.37256663e-04, -9.98722132e-03,  9.16488848e-03,
         1.23624732e-03,  6.16934287e-03, -1.01781502e-03,  2.26579802e-04,
        -8.53127582e-03,  5.41751538e-03,  8.42231145e-03, -9.81994389e-03,
        -6.47190204e-03,  3.45426398e-03,  7.26497597e-03,  5.73572752e-04,
        -3.79495516e-04,  3.20711144e-03,  1.40333265e-03,  5.67002471e-03,
        -2.19794004e-03, -3.00387444e-03, -6.42484571e-03, -5.10354813e-03,
        -8.27921308e-03, -3.34882568e-03, -9.92115875e-04,  8.59856070e-03,
         9.21170464e-03,  9.17586625e-03, -3.88124728e-03, -8.21228118e-03,
        -7.77879983e-03, -4.72730843e-03, -7.29832598e-03, -7.76984730e-03,
        -2.35701367e-03, -8.00967292e-03,  7.06535641e-03,  5.36315955e-03,
        -9.10047111e-03,  2.30228693e+00,  2.28617233e+00,  4.45393922e-02,
        -6.46873597e-04,  7.82475702e-02,  1.81646205e-02, -1.67780117e-01,
   

In [4]:
info

{'x_position': np.float64(0.004285169441919876),
 'y_position': np.float64(0.00480598111663944),
 'tendon_length': array([-0.01824226,  0.00124439]),
 'tendon_velocity': array([-0.00257102, -0.01305711]),
 'distance_from_origin': np.float64(0.0064389542349250145)}

As we can see, the humanoid environment gives us a TON of observations! We get dozens of variables representing various positions and velocities of body parts, the center of mass, and a lot of other variables I hardly understand. For an exhaustive list, see the [doc](https://gymnasium.farama.org/environments/mujoco/humanoid/#observation-space). The fact that there are so many variables here is what makes learning latent observations so obvious!

We also get some nice summary stats in info, but we aren't going to include them in our scrape.

In [5]:
def collect_rollout_data(env_name: str, out_dir: str, n_timesteps: int=10000, print_n_episodes: int=1000):
    """Simulates `n_timesteps` in the `env_name` environment, saving observations, rewards, and actions to a triplet of .npy files at `out_dir`."""
    env = gym.make(env_name, render_mode='rgb_array')
    obs, info = env.reset()
    observations, rewards, actions, done = [], [] , [], []
    episode_count = 0

    for timestep in range(n_timesteps):  # Run for n_timesteps or until the episode ends
        action = env.action_space.sample() #select random action
        obs, reward, terminated, truncated, info = env.step(action) #execute and get results
        observations.append(obs) #save observation
        rewards.append(reward) #save reward
        actions.append(action) #save action
        done.append(terminated or truncated) #save timestep of each episode's boundary
        if terminated or truncated: #check for game over, if so reset env
            episode_count+=1
            if episode_count % print_n_episodes == 0: print(f"finished {episode_count} episodes") #provide update on training
            observation, info = env.reset()
        env.close()
    np_obs, np_rewards, np_actions, np_done = np.array(observations), np.array(rewards), np.array(actions), np.array(done)
    print(f"observations has shape: {np_obs.shape}\trewards has shape: {np_rewards.shape}\tactions has shape: {np_actions.shape}\tdone has shape: {np_done.shape}")
    os.mkdir(f'{out_dir}/{env_name}_{n_timesteps}')
    np.save(f'{out_dir}/{env_name}_{n_timesteps}/observations.npy', np_obs) #load with: new_obs = np.load("../data/processed/Humanoid-v5_10000_rollout_observations.npy")
    np.save(f'{out_dir}/{env_name}_{n_timesteps}/rewards.npy', np_rewards)
    np.save(f'{out_dir}/{env_name}_{n_timesteps}/actions.npy', np_actions)
    np.save(f'{out_dir}/{env_name}_{n_timesteps}/done.npy', np_done)
    return np_obs, np_rewards, np_actions, np_done

In [6]:
humanoid_obs, humanoid_rewards, humanoid_actions, humanoid_done = collect_rollout_data('Humanoid-v5', "../data/processed", 10000, 100)

finished 100 episodes
finished 200 episodes
finished 300 episodes
finished 400 episodes
observations has shape: (10000, 348)	rewards has shape: (10000,)	actions has shape: (10000, 17)	done has shape: (10000,)


In [7]:
humanoid_rewards

array([4.88918423, 4.89604958, 4.90176838, ..., 4.90947515, 4.90974383,
       4.89052287])

In [8]:
humanoid_actions

array([[-0.01211881, -0.12276036, -0.07241314, ...,  0.18449783,
        -0.3750561 ,  0.35360506],
       [ 0.37878543,  0.04735717,  0.06504802, ..., -0.35926548,
         0.0173445 ,  0.0967169 ],
       [ 0.21681017, -0.03310588, -0.22394632, ...,  0.13517708,
         0.29709256,  0.01220466],
       ...,
       [-0.11548416,  0.06021108, -0.24303201, ...,  0.29784703,
         0.3527969 ,  0.0058307 ],
       [-0.23441796,  0.13045473, -0.00081047, ..., -0.21562773,
         0.39298594,  0.352991  ],
       [ 0.19539085,  0.38917974,  0.3781217 , ..., -0.28425267,
         0.3122244 ,  0.16986099]], dtype=float32)

In [9]:
humanoid_done, humanoid_done.sum()

(array([False, False, False, ..., False, False, False]), np.int64(407))

## Training the VAE

Now that we have an easy function for gathering experiences in the environment, we need to train the VAE module of our world model which will compress the observation space into latent space with fewer dimensions.

Note that the original World Model implementation worked with tensorflow 1.18.0. This is incredibly outdated (worked with Python 3.5), so let's get a newer version (tensorflow 2.19.0). Additionally, we need to change the structure of the VAE from working with images to working with a vector of observation data! Luckily, ChatGPT is very good at updating code (or it will be very obvious if it is not!).

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, Model, saving

@saving.register_keras_serializable()
class MLPVAE(Model):
    def __init__(self, input_dim=348, z_size=32, kl_tolerance=0.5):
        super(MLPVAE, self).__init__()
        self.z_size = z_size
        self.kl_tolerance = kl_tolerance

        # Encoder
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=(input_dim,)),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(2 * z_size),  # output both mu and logvar
        ])

        # Decoder
        self.decoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=(z_size,)),
            layers.Dense(128, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(input_dim, activation='linear'),  # output same shape as input
        ])

    def sample_z(self, mu, logvar):
        eps = tf.random.normal(shape=tf.shape(mu))
        sigma = tf.exp(0.5 * logvar)
        return mu + sigma * eps

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = tf.split(h, num_or_size_splits=2, axis=1)
        logvar = tf.clip_by_value(logvar, -10.0, 10.0)  # helps with exploding values
        return mu, logvar

    def decode(self, z):
        return self.decoder(z)

    def call(self, x):
        mu, logvar = self.encode(x)
        z = self.sample_z(mu, logvar)
        x_recon = self.decode(z)
        return x_recon, mu, logvar

    def compute_loss(self, x):
        x_recon, mu, logvar = self(x)
        recon_loss = tf.reduce_mean(tf.reduce_sum(tf.square(x - x_recon), axis=1))
        kl_loss = -0.5 * tf.reduce_sum(1 + logvar - tf.square(mu) - tf.exp(logvar), axis=1)
        kl_loss = tf.maximum(kl_loss, self.kl_tolerance * self.z_size)
        kl_loss = tf.reduce_mean(kl_loss)
        total_loss = recon_loss + kl_loss
        return total_loss, recon_loss, kl_loss

2025-05-10 12:53:45.742981: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-10 12:53:45.768872: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746896025.790914   21466 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746896025.797031   21466 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746896025.811468   21466 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Let's also write a training function for convenience.

In [11]:
def create_dataset(x_train, batch_size=64, shuffle_buffer=10000):
    # Assuming x_train is a NumPy array of shape [n_samples, 348]
    dataset = tf.data.Dataset.from_tensor_slices(x_train.astype(np.float32))
    dataset = dataset.shuffle(shuffle_buffer).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

def train_vae(model, dataset, epochs=10, learning_rate=1e-4):
    optimizer = tf.keras.optimizers.Adam(learning_rate)

    for epoch in range(epochs):
        total_loss = 0.0
        total_batches = 0
        for x_batch in dataset:
            with tf.GradientTape() as tape:
                loss, recon_loss, kl_loss = model.compute_loss(x_batch)
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            total_loss += loss.numpy()
            total_batches += 1

        avg_loss = total_loss / total_batches
        print(f"Epoch {epoch+1}: avg loss = {avg_loss:.4f}")


In [12]:
# x_train should be a NumPy array of shape (n_samples, 348)
x_train = humanoid_obs
x_train = (x_train - np.mean(x_train, axis=0)) / (np.std(x_train, axis=0) + 1e-6)

dataset = create_dataset(humanoid_obs, batch_size=64)
vae = MLPVAE(input_dim=348, z_size=32)
train_vae(vae, dataset, epochs=20)

2025-05-10 12:53:53.952050: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/seansteinle/.pyenv/versions/cs757-final-project/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
2025-05-10 12:54:14.168318: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1: avg loss = 294404.4062


2025-05-10 12:54:34.243369: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2: avg loss = 124580.4062
Epoch 3: avg loss = 66752.4688


2025-05-10 12:55:23.387848: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 4: avg loss = 57464.8281
Epoch 5: avg loss = 52356.0273
Epoch 6: avg loss = 47033.0430
Epoch 7: avg loss = 41048.5000


2025-05-10 12:56:45.393300: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 8: avg loss = 36293.6836
Epoch 9: avg loss = 32545.5039
Epoch 10: avg loss = 28815.9082
Epoch 11: avg loss = 25161.7969
Epoch 12: avg loss = 21866.2051
Epoch 13: avg loss = 19387.0508
Epoch 14: avg loss = 17487.5430
Epoch 15: avg loss = 15973.7705


2025-05-10 12:59:21.870055: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 16: avg loss = 14745.2646
Epoch 17: avg loss = 13713.4287
Epoch 18: avg loss = 12851.3877
Epoch 19: avg loss = 12226.2646
Epoch 20: avg loss = 11608.0508


Loss is going down! At first I got a tons of NAN values, but it's because I wasn't normalizing the input data and I also needed to clip the logvar values we were getting as a result of the encoding process. If you get NANs again, a lower learning rate could help too. Onto saving the model!

In [13]:
vae.save_weights('../models/vae/humanoid_10000_vae_model.weights.h5') #save ONLY weights -- much simpler than serializing the entire object

In [14]:
new_vae = MLPVAE(input_dim=348, z_size=32) #instantiate new model object 
new_vae(tf.zeros((1, 348))) #invoke it to build its shape 
new_vae.load_weights('../models/vae/humanoid_10000_vae_model.weights.h5') #now load weights into empty vector

In [15]:
new_vae

<MLPVAE name=mlpvae_1, built=True>

In [16]:
vae

<MLPVAE name=mlpvae, built=True>

## Training the MDN-RNN

Now that we have a model which captures observations, we're theoretically ~1/3 done with the project! I say theoretically because this was probably the easiest part of the project. Now onto the meat of world models: capturing the transitions of our environment and training the MDN-RNN!

### Prepping Rollout Data for the MDN-RNN

To train the MDN-RNN, we first need to enhance our basic rollout dataset with predictions of `mu` and `logvar` for each experience. Then we'll feed this information to the MDN-RNN.

In [17]:
#We can use the dataset records still in memory. Note that we only need the observations for now!
humanoid_obs.shape, humanoid_rewards.shape, humanoid_actions.shape, humanoid_done.shape

((10000, 348), (10000,), (10000, 17), (10000,))

In [18]:
#We can also use the VAE still in memory!
vae

<MLPVAE name=mlpvae, built=True>

In [19]:
humanoid_obs[0]

array([ 1.39437816e+00,  9.99968250e-01,  3.79864485e-03,  6.59385841e-03,
       -2.36428642e-03, -6.44013275e-03,  4.04482542e-02, -1.73932945e-02,
        7.02069696e-03, -1.64809519e-02, -5.87085982e-02, -8.31239564e-02,
       -2.32992584e-02,  5.96162997e-02, -3.89616689e-02,  2.65256705e-03,
       -8.19216933e-03, -1.37551596e-02,  1.69978972e-02, -4.73424357e-03,
       -5.79563778e-03,  2.03695153e-02,  2.00603784e-01,  4.37858659e-02,
       -2.19800178e-01,  1.58938603e-01,  4.42704435e-02,  1.07000006e+00,
       -1.31639174e+00,  2.79643428e+00, -1.52624409e+00,  1.82807029e+00,
       -2.23633396e+00, -7.54125526e+00, -1.01836506e+01, -1.99326228e+00,
        5.13092967e+00, -4.59377304e+00, -5.50918487e-01, -1.96932012e+00,
       -1.40859456e+00,  2.99984654e+00, -1.34652410e-01, -1.24951469e+00,
        1.88820864e+00,  2.30075750e+00,  2.28182635e+00,  4.14710432e-02,
       -9.92898973e-05,  1.05918959e-02,  7.30889934e-03, -3.23869504e-02,
       -1.31595755e-02,  

In [20]:
#That's it! Just predict for our entire observation set.
mu, logvar = vae.encode(humanoid_obs)
humanoid_z = vae.sample_z(mu, logvar).numpy()
humanoid_z.shape

(10000, 32)

In [21]:
np.save(f'../data/processed/Humanoid-v5_10000/z.npy', humanoid_z)

In [22]:
#We can use the dataset records still in memory. Note that we only need the observations for now!
humanoid_obs.shape, humanoid_rewards.shape, humanoid_actions.shape, humanoid_done.shape, humanoid_z.shape

((10000, 348), (10000,), (10000, 17), (10000,), (10000, 32))

### Core Training

Now that we have our mu and logvar arrays and also episode-wise aggregations of all of our data, we can train our MDN-RNN!

In [23]:
#CORE MODEL CLASS
class MDNRNN(tf.keras.Model):
    def __init__(self, latent_dim, action_dim, hidden_dim=256, num_mixtures=5):
        super().__init__()
        self.latent_dim = latent_dim
        self.action_dim = action_dim
        self.input_dim = latent_dim + action_dim
        self.hidden_dim = hidden_dim
        self.num_mixtures = num_mixtures

        # LSTM
        self.lstm = layers.LSTM(hidden_dim, return_sequences=True, return_state=True)

        # MDN output: means, stddevs, and mixture weights for latent prediction
        self.mdn_dense = layers.Dense(num_mixtures * (2 * latent_dim + 1))

        # Predict reward (scalar)
        self.reward_dense = layers.Dense(1)

        # Predict done (binary classification)
        self.done_dense = layers.Dense(1, activation="sigmoid")

    def call(self, inputs, initial_state=None, training=False):
        """
        inputs: (batch, seq_len, latent_dim + action_dim)
        """
        lstm_out, h, c = self.lstm(inputs, initial_state=initial_state, training=training)

        mdn_out = self.mdn_dense(lstm_out)
        reward_pred = self.reward_dense(lstm_out)
        done_pred = self.done_dense(lstm_out)

        return mdn_out, reward_pred, done_pred, [h, c]

    def get_mdn_params(self, mdn_out):
        """Split MDN output into pi, mu, sigma."""
        out = tf.reshape(mdn_out, [-1, self.num_mixtures, 2 * self.latent_dim + 1])
        pi = out[:, :, 0]
        mu = out[:, :, 1 : 1 + self.latent_dim]
        log_sigma = out[:, :, 1 + self.latent_dim :]
        sigma = tf.exp(log_sigma)

        pi = tf.nn.softmax(pi, axis=-1)  # mixture weights
        return pi, mu, sigma


In [24]:
#LOSS IMPLEMENTATION
import tensorflow_probability as tfp

def mdn_loss(z_target, pi, mu, sigma, eps=1e-8):
    """
    z_target: [batch * seq_len, latent_dim]
    pi: [batch * seq_len, num_mixtures]
    mu: [batch * seq_len, num_mixtures, latent_dim]
    sigma: [batch * seq_len, num_mixtures, latent_dim]
    """
    # Expand target for broadcasting: [batch, 1, latent_dim]
    z_expanded = tf.expand_dims(z_target, axis=1)

    # Create component Gaussians
    normal_dist = tfp.distributions.Normal(loc=mu, scale=sigma)
    log_probs = normal_dist.log_prob(z_expanded)  # shape: [batch, num_mixtures, latent_dim]

    # Sum over latent_dim: total log prob of each mixture component
    log_probs = tf.reduce_sum(log_probs, axis=-1)  # shape: [batch, num_mixtures]

    # Weight by mixture coefficients
    weighted_log_probs = log_probs + tf.math.log(pi + eps)  # log(pi * P)
    
    # LogSumExp over mixture components to marginalize
    log_likelihood = tf.reduce_logsumexp(weighted_log_probs, axis=-1)  # shape: [batch]

    # Negative log-likelihood
    return -tf.reduce_mean(log_likelihood)

def combined_loss(z_target, pi, mu, sigma, reward_target, reward_pred, done_target, done_pred,
                  reward_weight=1.0, done_weight=1.0):
    loss_mdn = mdn_loss(z_target, pi, mu, sigma)
    loss_reward = tf.reduce_mean(tf.square(reward_target - reward_pred))
    loss_done = tf.reduce_mean(tf.keras.losses.binary_crossentropy(done_target, done_pred))

    return loss_mdn + reward_weight * loss_reward + done_weight * loss_done


In [25]:
#TRAINING FUNCTION
def train_mdnrnn(model, dataset, epochs=10, learning_rate=1e-4, reward_weight=1.0, done_weight=1.0):
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    
    for epoch in range(epochs):
        total_loss = 0.0
        total_mdn_loss = 0.0
        total_reward_loss = 0.0
        total_done_loss = 0.0
        total_batches = 0

        for z_action, (z_next, reward, done) in dataset:
            # Flatten z_action into shape (batch, seq_len, latent_dim + action_dim)
            with tf.GradientTape() as tape:
                # Forward pass through the model
                mdn_out, reward_pred, done_pred, _ = model(z_action, training=True)
                pi, mu, sigma = model.get_mdn_params(mdn_out)

                # Compute MDN, reward, and done losses
                loss_mdn = mdn_loss(tf.reshape(z_next, [-1, model.latent_dim]), pi, mu, sigma)
                loss_reward = tf.reduce_mean(tf.square(reward - reward_pred))
                loss_done = tf.reduce_mean(tf.keras.losses.binary_crossentropy(done, done_pred))

                # Total loss
                total_loss = loss_mdn + reward_weight * loss_reward + done_weight * loss_done

            # Compute gradients and apply
            grads = tape.gradient(total_loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            # Accumulate the loss values
            total_loss += total_loss.numpy()
            total_mdn_loss += loss_mdn.numpy()
            total_reward_loss += loss_reward.numpy()
            total_done_loss += loss_done.numpy()

            total_batches += 1

        # Compute and print average losses for the epoch
        avg_loss = total_loss / total_batches
        avg_mdn_loss = total_mdn_loss / total_batches
        avg_reward_loss = total_reward_loss / total_batches
        avg_done_loss = total_done_loss / total_batches

        print(f"Epoch {epoch+1}:")
        print(f"  avg loss = {avg_loss:.4f}, mdn_loss = {avg_mdn_loss:.4f}, "
              f"reward_loss = {avg_reward_loss:.4f}, done_loss = {avg_done_loss:.4f}")


In [26]:
#DATA HANDLING
def preprocess_data(observations, actions, rewards, dones, sequence_length):
    """
    Yields tuples of the form:
    z_action: (sequence_length, latent_dim + action_dim)
    targets:  (z_next, reward, done), each of shape (sequence_length, ...)
    """
    for i in range(len(observations) - sequence_length):
        z_seq = observations[i:i+sequence_length]
        a_seq = actions[i:i+sequence_length]
        z_action = np.concatenate([z_seq, a_seq], axis=-1).astype(np.float32)

        z_next = observations[i+1:i+sequence_length+1].astype(np.float32)
        reward = rewards[i+1:i+sequence_length+1].astype(np.float32)
        done = dones[i+1:i+sequence_length+1].astype(np.float32)

        yield (
            z_action.astype(np.float32),
            (
                z_next.astype(np.float32),
                reward[:, None].astype(np.float32),  # <–– reshape from (T,) to (T,1)
                done[:, None].astype(np.float32)     # <–– reshape from (T,) to (T,1)
            )
        )


sequence_length = 10
latent_dim = 32
action_dim = 17
batch_size = 32

train_dataset = tf.data.Dataset.from_generator(
    lambda: preprocess_data(humanoid_z, humanoid_actions, humanoid_rewards, humanoid_done, sequence_length),
    output_signature=(
        tf.TensorSpec(shape=(sequence_length, latent_dim + action_dim), dtype=tf.float32),
        (
            tf.TensorSpec(shape=(sequence_length, latent_dim), dtype=tf.float32),
            tf.TensorSpec(shape=(sequence_length, 1), dtype=tf.float32),
            tf.TensorSpec(shape=(sequence_length, 1), dtype=tf.float32),
        )
    )
)

train_dataset = train_dataset.batch(batch_size).shuffle(1000)

In [27]:
mdnrnn = MDNRNN(latent_dim=32, action_dim=17)
train_mdnrnn(mdnrnn, train_dataset)

2025-05-10 13:01:26.248475: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:8: Filling up shuffle buffer (this may take a while): 192 of 1000
2025-05-10 13:01:32.488861: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.


Epoch 1:
  avg loss = 1.6931, mdn_loss = 894.2740, reward_loss = 13.1924, done_loss = 0.3610


2025-05-10 13:04:03.440020: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:8: Filling up shuffle buffer (this may take a while): 188 of 1000
2025-05-10 13:04:10.080250: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.


Epoch 2:
  avg loss = 1.3791, mdn_loss = 189.5247, reward_loss = 3.1045, done_loss = 0.1949


2025-05-10 13:06:29.482147: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:8: Filling up shuffle buffer (this may take a while): 200 of 1000
2025-05-10 13:06:35.321125: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.


...

Epoch 8:
  avg loss = 0.8353, mdn_loss = 125.5876, reward_loss = 1.4757, done_loss = 0.1725


2025-05-10 13:20:50.361600: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:8: Filling up shuffle buffer (this may take a while): 177 of 1000
2025-05-10 13:20:57.826262: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.


Epoch 9:
  avg loss = 0.8084, mdn_loss = 116.8234, reward_loss = 1.4340, done_loss = 0.1727


2025-05-10 13:23:46.273652: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:8: Filling up shuffle buffer (this may take a while): 163 of 1000
2025-05-10 13:23:54.390910: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.


Epoch 10:
  avg loss = 0.7236, mdn_loss = 115.2999, reward_loss = 1.3894, done_loss = 0.1717


## Training the Controller

Now that we have a VAE and an MDN-RNN that are (presumably) learning, let's write the final piece of the puzzle -- the MDN-RNN!



In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, saving
import gymnasium as gym

# --- Controller model ---
@saving.register_keras_serializable()
class LinearController(Model):
    def __init__(self, input_dim, action_dim):
        super(LinearController, self).__init__()
        self.linear = layers.Dense(action_dim, activation='tanh', use_bias=False)
        self.linear(tf.zeros((1, input_dim)))  # force build so weights are initialized

    def call(self, inputs):
        return self.linear(inputs)

    def get_weights_flat(self):
        return tf.reshape(self.linear.kernel, [-1])

    def set_weights_flat(self, flat_weights):
        new_weights = tf.reshape(flat_weights, self.linear.kernel.shape)
        self.linear.kernel.assign(new_weights)

# --- Evolution strategy ---
def evolve_controller(controller, vae, mdn_rnn, env, generations=10, pop_size=64, sigma=0.1, elite_frac=0.2):
    input_dim = controller.linear.kernel.shape[0]
    action_dim = controller.linear.kernel.shape[1]
    weight_dim = input_dim * action_dim

    base_weights = controller.get_weights_flat().numpy()
    elite_num = max(1, int(pop_size * elite_frac))

    for gen in range(generations):
        population = [base_weights + sigma * np.random.randn(weight_dim) for _ in range(pop_size)]
        scores = []

        for i, individual in enumerate(population):
            controller.set_weights_flat(individual)
            reward = evaluate_controller(controller, vae, mdn_rnn, env)
            scores.append((reward, individual))

        scores.sort(key=lambda x: -x[0])
        elites = [w for _, w in scores[:elite_num]]
        new_mean = np.mean(elites, axis=0)
        base_weights = new_mean

        print(f"Gen {gen+1}: Best score = {scores[0][0]:.2f}")

    controller.set_weights_flat(base_weights)
    return controller

In [29]:
def evaluate_controller(controller, vae, mdn_rnn, env, max_steps=1000):
    total_reward = 0
    obs, _ = env.reset()
    done = False
    h = tf.zeros((1, 256))  # RNN hidden state (adjust as needed)

    state = [h, h]  # Initialize state for MDN-RNN (h, c)
    for step in range(max_steps):
        x = tf.convert_to_tensor(obs[None, :], dtype=tf.float32)
        mu, _ = vae.encode(x)
        z = mu  # Use the mean (mu) as the latent vector

        zh = tf.concat([z, h], axis=1)  # Concatenate latent vector and hidden state
        action = controller(zh).numpy()[0]  # Get action from controller

        # Step in the environment
        obs, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        # Update hidden state using the MDN-RNN
        rnn_input = tf.concat([z, action[None]], axis=1)  # (latent_dim + action_dim)
        rnn_input = tf.expand_dims(rnn_input, axis=1)  # Shape becomes (1, 1, 50)

        mdn_out, reward_pred, done_pred, state = mdn_rnn(rnn_input, initial_state=state)

        total_reward += reward

        # Optionally: You can use reward_pred and done_pred for monitoring

        if done:
            break

    return total_reward

In [30]:
z_dim = 32
h_dim = 256
action_dim = env.action_space.shape[0]

controller = LinearController(input_dim=z_dim + h_dim, action_dim=action_dim)

# Assume `vae` and `mdn_rnn` are your pretrained models
trained_controller = evolve_controller(controller, vae, mdnrnn, env)

Gen 1: Best score = 255.17
Gen 2: Best score = 229.66
Gen 3: Best score = 255.10
Gen 4: Best score = 385.72
Gen 5: Best score = 295.82
Gen 6: Best score = 375.04
Gen 7: Best score = 288.08
Gen 8: Best score = 383.42
Gen 9: Best score = 489.99
Gen 10: Best score = 437.53


So right now, controller with JUST VAE is working. Also, we're missing the CMA-ES piece! We just do random search optimization right now.

## Early Results

With a very basic model trained, we can now compare the performance of our model with a random policy and a PPO policy. To do this, we'll have to wrap our model up in a nice interface that plays well with our `play_agent()` function from `./gymnasium_basics.ipynb`.

In [31]:
def make_world_model_policy(vae, mdn_rnn, controller):
    h = tf.zeros((1, 256))
    state = [h, h]

    def policy(obs):
        nonlocal state, h
        x = tf.convert_to_tensor(obs[None, :], dtype=tf.float32)
        mu, _ = vae.encode(x)
        z = mu

        zh = tf.concat([z, h], axis=1)
        action = controller(zh).numpy()[0]

        rnn_input = tf.concat([z, action[None]], axis=1)
        rnn_input = tf.expand_dims(rnn_input, axis=1)  # shape: (1, 1, 50)
        _, _, _, state = mdn_rnn(rnn_input, initial_state=state)
        h = state[0]

        return action

    return policy

In [32]:
#a slightly updated version of play_agent()
def play_agent(env_name, model=None, n_timesteps=50, render_mode="human"):
    """Plays games using a provided model or random policy."""
    env = gym.make(env_name, render_mode=render_mode)
    obs, info = env.reset()
    rewards = []
    episode_lengths = []
    steps = 0

    while steps < n_timesteps:
        env.render()
        if model is None: #random policy
            action = env.action_space.sample()
        elif hasattr(model, "predict"): #SB3 models use 'predict'
            action, _ = model.predict(obs)
        else: #our model uses `model(obs)`
            action = model(obs)  # assumes model is a callable function

        obs, reward, terminated, truncated, info = env.step(action)
        rewards.append(reward)
        steps += 1

        if terminated or truncated:
            episode_lengths.append(steps)
            obs, info = env.reset()

    env.close()
    return rewards, episode_lengths

In [33]:
wm_policy = make_world_model_policy(vae, mdnrnn, trained_controller)
rewards, lengths = play_agent(env_name="Humanoid-v5", model=wm_policy, n_timesteps=1000)
np.sum(rewards), np.mean(lengths) #avg reward, avg episode length of world model policy

/home/seansteinle/.pyenv/versions/cs757-final-project/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


(np.float64(3591.5168913059815), np.float64(516.0))

In [34]:
np.sum(rewards), np.mean(lengths)

(np.float64(3591.5168913059815), np.float64(516.0))

In [35]:
rewards, lengths = play_agent(env_name="Humanoid-v5", model=None, n_timesteps=1000)
np.sum(rewards), np.mean(lengths) #avg reward, avg episode length of random policy

(np.float64(4653.932537313411), np.float64(510.9))

Okay, world models is still doing worse than random. Tough times! That said, we haven't done the magic of **scaling** yet! Let's break this bad boy up into a few scripts and hit the HPC!

In [38]:
mdnrnn.save_weights('../models/mdnrnn/humanoid_10000_mdnrnn_model.weights.h5') #save ONLY weights -- much simpler than serializing the entire object

In [43]:
new_mdnrnn = mdnrnn = MDNRNN(latent_dim=32, action_dim=17) #instantiate new model object 
new_mdnrnn(tf.zeros((1, 10, 49))) #invoke it to build its shape 
new_mdnrnn.load_weights('../models/mdnrnn/humanoid_10000_mdnrnn_model.weights.h5') #now load weights into empty vector